<a href="https://colab.research.google.com/github/Sidharth-Saji/WildFire_Monitoring_Using_Satellite_Image/blob/main/Model_Training_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import albumentations as A

## Creating Dataframes

In [ ]:
def create_df_img(filepath):
    """ Create a DataFrame with the filepath and the labels of the pictures
    
    Args : filepath of the images
    Return : pandas dataframe     
    """
    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepath))
    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    df = pd.concat([filepath, labels], axis=1)
    df = df.sample(frac=1).reset_index(drop = True)
    return df

In [ ]:
train_dir = Path('train')
filepaths = list(train_dir.glob(r'**/*.jpg'))
val_dir = Path('train')
filepaths_valid = list(val_dir.glob(r'**/*.jpg'))

### Checking if dataframe loaded correctly

In [ ]:
df_train = create_df_img(filepaths)
df_valid = create_df_img(filepaths_valid)
df_train.head()
df_valid.head()

,Filepath,Label
0,"train\nowildfire\-73.89897,45.474524.jpg",nowildfire
1,"train\nowildfire\-73.444224,45.585472.jpg",nowildfire
2,"train\nowildfire\-73.465388,45.584881.jpg",nowildfire
3,"train\nowildfire\-73.455049,45.552418.jpg",nowildfire
4,"train\nowildfire\-73.433861,45.472406.jpg",nowildfire


#DATA AUGMENTATION 

In [ ]:
def augmentation_image(image):
    """
    Function for Augmenting Images
    """
    aug = A.Compose([
        A.Flip(),
        A.Rotate(limit=75,always_apply=True),
    ])
    augmented_image = aug(image=image)['image']
    return augmented_image

In [ ]:
BATCH_SIZE = 256
# Creating Train Generator using Image Generator
train_datagenerator = ImageDataGenerator(dtype='float32', rescale= 1./255., preprocessing_function=augmentation_image)
train_generator = train_datagenerator.flow_from_dataframe(df_train,
                                                    x_col='Filepath',
                                                    y_col='Label',
                                                    shuffle=True,
                                                    batch_size = BATCH_SIZE,
                                                    target_size = (350,350),
                                                    class_mode = 'categorical')

#Creating Validity Checker
val_datagen = ImageDataGenerator(dtype='float32', rescale= 1./255.)
val_generator = val_datagen.flow_from_dataframe(df_valid,
                                                    x_col='Filepath',
                                                    y_col='Label',
                                                    shuffle=True,
                                                    batch_size = BATCH_SIZE,
                                                    target_size = (350,350),
                                                    class_mode = 'categorical')

Found 206 validated image filenames belonging to 2 classes.
Found 206 validated image filenames belonging to 2 classes.


# MODEL BUILDING

In [ ]:
shrink = 1e-3 
image_shape=plt.imread(df_train["Filepath"][0]).shape
model = Sequential([
    Conv2D(filters = 8 , kernel_size = 2, activation = 'relu', 
    input_shape = image_shape), MaxPooling2D(pool_size = 2),
    
    Conv2D(filters = 16 , kernel_size = 2, activation = 'relu', 
    input_shape = image_shape), MaxPooling2D(pool_size = 2),
    
    Conv2D(filters = 32 , kernel_size = 2, activation = 'relu',
           kernel_regularizer = regularizers.l2(shrink)),
    MaxPooling2D(pool_size = 2),
    
    
    Flatten(),
    Dense(300,activation='relu'),
    Dropout(0.5),
    Dense(100,activation='relu'),
    Dropout(0.4),
    Dense(2,activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 349, 349, 8)       104       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 174, 174, 8)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 173, 173, 16)      528       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 86, 86, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 85, 85, 32)        2080      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 42, 42, 32)       0

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

checkpointer = ModelCheckpoint('model.hdf5',verbose=1, save_best_only= True)

early_stopping = EarlyStopping(monitor= 'val_loss', patience= 10)

optimizer = optimizers.Adam(learning_rate= 0.00001, weight_decay= 1e-5)

checkpointer = ModelCheckpoint('model.hdf5',verbose=1, save_best_only= True)

early_stopping = EarlyStopping(monitor= 'val_loss', patience= 10)

optimizer = optimizers.Adam(learning_rate= 0.00001, weight_decay= 1e-5)

model.compile(loss= 'categorical_crossentropy', optimizer= 'adam',
                    metrics=['AUC','acc'])


history = model.fit(train_generator,
                    epochs = 100,
                    verbose = 1,
                    validation_data = val_generator,
                    callbacks = [checkpointer, early_stopping])

1/1 [==============================] - ETA: 0s - loss: 2.0095 - auc: 0.5683 - acc: 0.5777
Epoch 1: val_loss improved from inf to 5.58305, saving model to model.hdf5
1/1 [==============================] - 19s 19s/step - loss: 2.0095 - auc: 0.5683 - acc: 0.5777 - val_loss: 5.5831 - val_auc: 0.5073 - val_acc: 0.5000


In [ ]:
model.save('WildFireDetector.h5')